In [1]:
# -*- coding: utf-8 -*-
"""
@author: Etienne Kras
"""

# generic imports
import sys
import os
import numpy as np
import geopandas as gpd
import time
import geemap
import geojson
import ee
ee.Initialize(project="bathymetry") # use the GEE project-id here

# specific imports
from typing import Any, Dict, List, Optional
from geojson import Feature, FeatureCollection, dump
from shapely.geometry import Polygon, MultiPolygon, shape
from dateutil.relativedelta import *
from google.cloud import storage
from logging import Logger, getLogger
from googleapiclient.discovery import build
from re import sub
from ctypes import ArgumentError
from functools import partial
from dateutil.parser import parse

# custom functionality import without requirement to pip install package
local_path = r"C:\Users\kras\Documents\GitHub\ee-packages-py"  # path to local GitHub clone
sys.path.append(local_path)
from eepackages.applications.bathymetry import Bathymetry
from eepackages import tiler

logger: Logger = getLogger(__name__)

C:\Users\kras\AppData\Local\Temp\ipykernel_11468\2621078122.py:10: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [2]:
# TODO: 
# look if scale / crs does not influence the output used before exporting as we have differences between the GEE export and the local post-processed export

# Project specific toggles

In [3]:
# acknowledgements & code references:
# https://github.com/openearth/eo-bathymetry/
# https://github.com/openearth/eo-bathymetry-functions/
# https://github.com/gee-community/ee-packages-py

In [13]:
# see scheme at https://github.com/openearth/eo-bathymetry/blob/master/notebooks/rws-bathymetry/acces_api.pdf for a workflow visualization 

# project toggles
main_fol = r"p:\11209821-cmems-global-sdb" # name of the main local folder 
bucket = "cmems-sdb" # name of the Google Cloud Storage bucket to store files in the cloud
credential_file = r"p:\11209821-cmems-global-sdb\00_miscellaneous\KEYS\bathymetry-543b622ddce7.json" # Cloud Storage credential key
output_fol = r"01_intertidal\02_data\04_calibrated" # name of the overall project
project_name = "AOI_GER_WaddenSea" # name of the project AoI
draw_AoI = 0 # toggle 1 to draw AoI, 0 to load

# composite image toggles
mode = "intertidal_improved_100m" # specify mode, either "intertidal" or "subtidal"
start_date = "2021-01-01" # start date of the composites
stop_date = "2022-01-01" # end date of the composites
compo_int = 12 # composite interval [months]
compo_len = 12 # composite length [months]
scale = 100  # output resolution of the image [m]
crs = "EPSG:4326" # output projection of the image

# tiling options
zoomed_list = [9, 10, 11] # list with zoom levels to be inspected
sel_tile = 2 # idx of chosen tile level in zoomed_list (inspect the map to chose it accordingly), z9 too big for in memory computations
# note, see https://www.openearth.nl/rws-bathymetry/2019.html; Z9 is optimal size..

# load google credentials, if specified
if not credential_file == "":  
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = str(credential_file)

# load GTSM & gebco data
gtsm_col = ee.FeatureCollection('projects/bathymetry/assets/gtsm_waterlevels')
gebco_image = ee.Image('projects/bathymetry/assets/gebco_2023_hat_lat')

# Pre-processing using the API

In [5]:
# DELETE LATER

# # open all AoIs in the folder
# allfiles = os.listdir(os.path.join(main_fol, "00_miscellaneous\AOIs"))
# features = []
# for i in allfiles:
#     if i.endswith(".geojson"):
#         print(i)
#         with open(os.path.join(main_fol, "00_miscellaneous\AOIs", i), 'r') as f:
#             data = geojson.load(f)
#             mp = MultiPolygon(shape(data["features"][0]['geometry']))
#             features.append(Feature(geometry=mp, properties={"name": i.split(".geojson")[0]}))

# # store the combined AoI
# feature_collection = FeatureCollection(features)
# feature_collection.crs = {"type": "name","properties": {"name": "epsg:4326"}} 
# with open(os.path.join(main_fol,"00_miscellaneous\AOIs", project_name + "geojson"), "w") as f: # geojson
#     dump(feature_collection, f)


In [6]:
# draw or load Area of Interest (AoI)

# TODO: take center of AOI input file if present, put in random coordinate and let user find a place and draw a polygon
# TODO: fix horizontal tiling error (DOS) in API (to use multiple tiles) or move to single polygon run if AoI crosses multiple tiles
Map = geemap.Map(center=(54.2, 6.7), zoom=8) # initialize map with base in Hudayriat

if draw_AoI == 1:
    print("Please draw a polygon somewhere in a water body") # identifier
if draw_AoI == 0:
    # open AoI
    print("Loading and visualizing AoI") #identifier
    #AoIee = geemap.geojson_to_ee(os.path.join(main_fol,'AOI',project_name+'.geojson'))

    with open(os.path.join(main_fol, "00_miscellaneous\AOIs", project_name + ".geojson"), 'r') as f:
        contents = geojson.loads(f.read())
    AoIee = ee.Geometry(contents["features"][0]["geometry"])

    Map.addLayer(AoIee, {}, "AoI")

Map # show map

Loading and visualizing AoI


Map(center=[54.2, 6.7], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(c…

In [7]:
# (re)construct the AoI

if draw_AoI == 1:
    
    print("Constructing AoI from drawn polygon") # identifier
    
    # get AoI 
    AoIee = ee.FeatureCollection(Map.draw_features) # make featurecollection
    AoI = Polygon(AoIee.getInfo()["features"][0]["geometry"]["coordinates"][0]) # create AoI shapefile

    # export AoI
    features = []
    features.append(Feature(geometry=AoI, properties={"AoI": project_name}))
    feature_collection = FeatureCollection(features)
    with open(os.path.join(main_fol,"AOI",project_name + ".geojson"), "w") as f: # geojson
        dump(feature_collection, f)
    gdr = gpd.GeoDataFrame({"properties":{"AoI": project_name}, "geometry": AoI}, crs="EPSG:4326") #shp
    gdr.to_file(os.path.join(main_fol,"AOI",project_name+".shp"))
    bounds = ee.Geometry.Polygon([[[a,b] for a, b in zip(*AoI.exterior.coords.xy)]])
    
if draw_AoI == 0:
    print("Reconstructing AoI from loaded file")
    # get AoI
    with open(os.path.join(main_fol,"00_miscellaneous\AOIs",project_name+".geojson")) as f:
        AoIjson = geojson.load(f)
    # try: # drawn polygon in this script
    #     AoI = Polygon(AoIjson["features"][0]["geometry"]["coordinates"]) 
    # except: # drawn in QGIS / ArcGIS and written to geojson there (client file)
    #     AoI = Polygon(AoIjson["features"][0]["geometry"]["coordinates"][0])
    # bounds = ee.Geometry.Polygon([[[a,b] for a, b in zip(*AoI.exterior.coords.xy)]])
    bounds = ee.Geometry.MultiPolygon(AoIjson["features"][0]["geometry"]["coordinates"])

    # make list of multipolygons a single one
    # MPL = []
    # for i in AoIjson["features"]:
    #     for j in i["geometry"]["coordinates"]:
    #         MPL.append(j)

    # bounds = ee.Geometry.MultiPolygon(MPL)

Reconstructing AoI from loaded file


In [8]:
# tiling the AoI, show on map to decide zoom level to go for

def add_tile_bounds(zoom):
    tiled = tiler.get_tiles_for_geometry(bounds, zoom)
    Map.addLayer(tiled.style(width=max(1, 10 - zoom), fillColor= "00000022"), {}, "tiles " + str(zoom))

    return(tiled)

tiles = list(map(add_tile_bounds, zoomed_list)) # add tiles for different zoom levels

In [9]:
# seleting the correct tiled AoI without showing it on a map 
# note, adjust sel_tile accordingly 
tiled = [tiles[sel_tile]]

In [11]:
# tiling multiple AoIs without showing it on a map 
# note, adjust sel_tile accordingly 

allfiles = os.listdir(os.path.join(main_fol, "00_miscellaneous\AOIs"))
tiled = []
for i in allfiles:
    if i.endswith(".geojson") and not "CMEMS" in i and not "adjusted" in i: #and ("GER" in i or "BRA" in i or "ZAF" in i):
        print(i)
        with open(os.path.join(main_fol,"00_miscellaneous\AOIs",i)) as f:
            AoIjson = geojson.load(f)
        # try: # drawn polygon in this script
        #     AoI = Polygon(AoIjson["features"][0]["geometry"]["coordinates"]) 
        # except: # drawn in QGIS / ArcGIS and written to geojson there (client file)
        #     AoI = Polygon(AoIjson["features"][0]["geometry"]["coordinates"][0])
        # bounds = ee.Geometry.Polygon([[[a,b] for a, b in zip(*AoI.exterior.coords.xy)]])
        bounds = ee.Geometry.MultiPolygon(AoIjson["features"][0]["geometry"]["coordinates"])

        tiled.append(tiler.get_tiles_for_geometry(bounds, zoomed_list[sel_tile]))

AOI_BRA_SaoPaulo.geojson
AOI_CAN_KingWilliamIsland.geojson
AOI_CAN_PrinceRupertIsland.geojson
AOI_GER_WaddenSea.geojson
AOI_IND_Okha.geojson
AOI_KOR_Tean.geojson
AOI_MYT_Mayotte.geojson
AOI_NCL_IleDePins.geojson
AOI_Vir_CruzBay.geojson
AOI_ZAF_WoodyCape.geojson


In [12]:
# export selected tiles (represented by zoom level) to geojsons
# note, adjust sel_tile accordingly 
# for tiles in tiled:
#     for idx, tile in enumerate(tiles.getInfo()["features"]):
#         tile_pol = Polygon(tile['geometry']['coordinates'][0]) # create tile polygon
#         tile_name = "z%s_x%s_y%s"%(tile["properties"]["zoom"], int(float(tile["properties"]["tx"])), int(float(tile["properties"]["ty"])))

#         features = []
#         features.append(Feature(geometry=tile_pol, properties={"name": tile_name}))
#         feature_collection = FeatureCollection(features)
#         feature_collection.crs = {"type": "name","properties": {"name": "epsg:3857"}} # default EE projection
#         with open(os.path.join(main_fol,"00_miscellaneous\AOI_polygons",tile_name + ".geojson"), "w") as f: # geojson
#             dump(feature_collection, f)

# Compute SDB using the API

In [14]:
# functionality for intertidal sdb to pre-filter tiles to be processed

# filter tiles close to GTSM
def filter_tiles_close_to_GTSM(gtsm_col, start_date, stop_date, tile=ee.Feature(None),
                              max_spatial_offset=1):
    
    # Get area around the tile
    #tile_centroid = ee.Geometry.centroid(tile.geometry(), maxError=1)
    tile_footprint = ee.Geometry(tile.geometry())
    tile_buffer = tile_footprint.buffer(max_spatial_offset*1000)

    # Get period around image time
    tile_time_start = ee.Date(start_date)
    tile_time_end = ee.Date(stop_date) 
    
    # Filter gtsm data based on tile footprint and period
    gtsm_col = gtsm_col.filterBounds(tile_buffer)
    gtsm_col = gtsm_col.filterDate(ee.Date(tile_time_start.millis()), ee.Date(tile_time_end.millis()))

    # Add GTSM size to image
    tile = tile.set("max_gtsm_size", gtsm_col.size())
    
    return tile

In [17]:
# functions to compute sub & intertidal bathymetry proxies based on standardized SlippyMap tiling practice
# functions taken from: https://github.com/openearth/eo-bathymetry/blob/master/notebooks/rws-bathymetry/export_bathymetry.ipynb
# resembles similar behaviour as in https://github.com/openearth/eo-bathymetry-functions but slightly adjusted for local study 

def get_tile_subtidal_bathymetry(tile: ee.Feature, start: ee.String, stop: ee.String) -> ee.Image:
    """
    Get subtidal bathymetry based on tile geometry.
    Server-side compliant for GEE.

    args:
        tile (ee.Feature): tile geometry used to obtain bathymetry.
        start (ee.String): start date in YYYY-MM-dd format.
        stop (ee.String): stop date in YYYY-MM-dd format.
    
    returns:
        ee.Image: image containing subtidal bathymetry covering tile.
    """

    bounds: ee.Geometry = ee.Feature(tile).geometry().bounds(1)
    sdb: Bathymetry = Bathymetry()
    zoom: ee.String = ee.String(tile.get("zoom"))
    tx: ee.String = ee.String(tile.get("tx"))
    ty: ee.String = ee.String(tile.get("ty"))
    tile_name: ee.String = ee.String("z").cat(zoom).cat("_x").cat(tx).cat("_y").cat(ty).replace("\.\d+", "", "g")
    img_fullname: ee.String = ee.String(tile_name).cat("_t").cat(ee.Date(start).millis().format())
        
    image: ee.Image = sdb.compute_inverse_depth(
                bounds=bounds,
                start=start,
                stop=stop,
                scale=tiler.zoom_to_scale(ee.Number.parse(tile.get("zoom"))).multiply(5), # scale to search for clean images
                missions=["S2", "L8"],
                filter_masked=True,
                skip_neighborhood_search=False
                # cloud_frequency_threshold_data=,
                # pansharpen=,
                # skip_scene_boundary_fix=,
                # bounds_buffer=
    ).clip(bounds)

    image = image.set(
        "fullname", img_fullname,
        "system:time_start", ee.Date(start).millis(),
        "system:time_stop", ee.Date(stop).millis(),
        "zoom", zoom,
        "tx", tx,
        "ty", ty
    )
    return image

def get_tile_intertidal_bathymetry(tile: ee.Feature, start: ee.String, stop: ee.String) -> ee.Image:
    """
    Get intertidal bathymetry based on tile geometry.
    Server-side compliant for GEE.

    args:
        tile (ee.Feature): tile geometry used to obtain bathymetry.
        start (ee.String): start date in YYYY-MM-dd format.
        stop (ee.String): stop date in YYYY-MM-dd format.
    
    returns:
        ee.Image: image containing intertidal bathymetry covering tile.
    """

    bounds: ee.Geometry = ee.Feature(tile).geometry().bounds(1)
    sdb: Bathymetry = Bathymetry()
    zoom: ee.String = ee.String(tile.get("zoom"))
    tx: ee.String = ee.String(tile.get("tx"))
    ty: ee.String = ee.String(tile.get("ty"))
    tile_name: ee.String = ee.String("z").cat(zoom).cat("_x").cat(tx).cat("_y").cat(ty).replace("\.\d+", "", "g")
    img_fullname: ee.String = ee.String(tile_name).cat("_t").cat(ee.Date(start).millis().format())
        
    image: ee.Image = sdb.compute_intertidal_depth(
        bounds=bounds,
        start=start,
        stop=stop,
        scale=tiler.zoom_to_scale(ee.Number.parse(tile.get("zoom"))).multiply(5), # scale to search for clean images
        # missions=['S2', 'L8'],
        # filter: ee.Filter.dayOfYear(7*30, 9*30), # summer-only
        filter_masked=False, 
        # filterMaskedFraction = 0.5,
        # skip_scene_boundary_fix=False,
        # skip_neighborhood_search=False,
        neighborhood_search_parameters={"erosion": 0, "dilation": 0, "weight": 50},
        bounds_buffer=0,
        water_index_min=-0.05,
        water_index_max=0.15,
        # lowerCdfBoundary=45,
        # upperCdfBoundary=50,
        # cloud_frequency_threshold_data=0.15, 
        clip = True
    )# .reproject(ee.Projection("EPSG:3857").atScale(90))

    image = image.set(
        "fullname", img_fullname,
        "system:time_start", ee.Date(start).millis(),
        "system:time_stop", ee.Date(stop).millis(),
        "zoom", zoom,
        "tx", tx,
        "ty", ty
    )

    return image

def tile_to_asset(
    image: ee.Image,
    tile: ee.Feature,
    export_scale: int,
    asset_path_prefix: str,
    asset_name: str,
    overwrite: bool
) -> Optional[ee.batch.Task]:
    
    asset_id: str = f"{asset_path_prefix}/{asset_name}"
    asset: Dict[str, Any] = ee.data.getInfo(asset_id)
    if overwrite and asset:
        logger.info(f"deleting asset {asset}")
        ee.data.deleteAsset(asset_id)
    elif asset:
        logger.info(f"asset {asset} already exists, skipping {asset_name}")
        return
    task: ee.batch.Task = ee.batch.Export.image.toAsset(
        image,
        assetId=asset_id,
        description=asset_name,
        region=tile.geometry(),
        scale=export_scale,
        maxPixels= 1e10
    )
    task.start()
    logger.info(f"exporting {asset_name} to {asset_id}")

def tile_to_cloud_storage(
    image: ee.Image,
    tile: ee.Feature,
    crs: str,
    export_scale: int,
    bucket: str,
    bucket_path: str,
    overwrite: bool
) -> Optional[ee.batch.Task]:
    with build('storage', 'v1') as storage:
        res = storage.objects().list(bucket=bucket, prefix="/".join(bucket_path.split("/")[:-1])).execute()
    if not overwrite:
        try:
            object_exists = any(map(lambda item: item.get("name").startswith(bucket_path), res.get("items")))
        except AttributeError:
            object_exists = False
        if object_exists:
            logger.info(f"object {bucket_path} already exists in bucket {bucket}, skipping")
            return
        
    task: ee.batch.Task = ee.batch.Export.image.toCloudStorage(
        image,
        bucket=bucket,
        description=bucket_path.replace("/", "_"),
        fileNamePrefix=bucket_path,
        region=tile.geometry(),
        scale=export_scale,
        crs=crs,
        fileFormat='GeoTIFF',
        formatOptions= {'cloudOptimized': True}, # enables easy QGIS plotting
        maxPixels= 1e10
    )
    task.start()
    return task

def export_sdb_tiles(
    sink: str,
    tile_list: ee.List,
    num_tiles: int,
    export_scale: int,
    crs: str,
    sdb_tiles: ee.ImageCollection,
    name_suffix: str,
    mode: str,
    task_list: List[ee.batch.Task],
    overwrite: bool,
    bucket: Optional[str] = None
) -> List[ee.batch.Task]:
    """
    Export list of tiled images containing sub or intertidal tidal bathymetry. Fires off the tasks and adds to the list of tasks.
    based on: https://github.com/gee-community/gee_tools/blob/master/geetools/batch/imagecollection.py#L166

    args:
        sink (str): type of data sink to export to. Viable options are: "asset" and "cloud".
        tile_list (ee.List): list of tile features.
        num_tiles (int): number of tiles in `tile_list`.
        scale (int): scale of the export product.
        sdb_tiles (ee.ImageCollection): collection of subtidal bathymetry images corresponding
            to input tiles.
        name_suffix (str): unique identifier after tile statistics.
        task_list (List[ee.batch.Task]): list of tasks, adds tasks created to this list.
        overwrite (bool): whether to overwrite the current assets under the same `asset_path`.
        bucket (str): Bucket where the data is stored. Only used when sink = "cloud"
    
    returns:
        List[ee.batch.Task]: list of started tasks

    """
    if sink == "asset":
        user_name: str = ee.data.getAssetRoots()[0]["id"].split("/")[-1]
        asset_path_prefix: str = f"users/{user_name}/eo-bathymetry"
        ee.data.create_assets(asset_ids=[asset_path_prefix], asset_type="Folder", mk_parents=True)
    
    for i in range(num_tiles):
        # get tile
        temp_tile: ee.Feature = ee.Feature(tile_list.get(i))
        tile_metadata: Dict[str, Any] = temp_tile.getInfo()["properties"]
        tx: str = tile_metadata["tx"]
        ty: str = tile_metadata["ty"]
        zoom: str = tile_metadata["zoom"]
        # filter imagecollection based on tile
        filtered_ic: ee.ImageCollection = sdb_tiles \
            .filterMetadata("tx", "equals", tx) \
            .filterMetadata("ty", "equals", ty) \
            .filterMetadata("zoom", "equals", zoom)
        # if filtered correctly, only a single image remains
        img: ee.Image = ee.Image(filtered_ic.first())  # have to cast here
        img_name: str = sub(r"\.\d+", "", f"{mode}/z{zoom}/x{tx}/y{ty}/") + name_suffix 
        print("Submitting task for tile: ", img_name)
        # Export images
        if sink == "asset":  # Replace with case / switch in python 3.10
            task: Optional[ee.batch.Task] = tile_to_asset(
                image=img,
                tile=temp_tile,
                export_scale=export_scale,
                asset_path_prefix=asset_path_prefix,
                asset_name=img_name.replace("/","_"),
                overwrite=overwrite
            )
            if task: task_list.append(task)
        elif sink == "cloud":
            if not bucket:
                raise ArgumentError("Sink option requires \"bucket\" arg.")
            task: ee.batch.Task = tile_to_cloud_storage(
                image=img,
                tile=temp_tile,
                export_scale=export_scale,
                crs=crs, 
                bucket=bucket,
                bucket_path=img_name,
                overwrite=overwrite
            )
        else:
            raise ArgumentError("unrecognized data sink: {sink}")
        task_list.append(task)
    return task_list

def export_tiles(
    sink: str,
    mode: str,
    geometry: ee.Geometry,
    zoom: int,
    start: str,
    stop: str,
    scale: Optional[float] = None,
    crs: str = "EPSG:4326",
    buf_pix: int = 0,
    step_months: int = 3,
    window_months: int = 24,
    overwrite: bool = False,
    bucket: Optional[str] = None
) -> None:
    """
    From a geometry, creates tiles of input zoom level, calculates subtidal bathymetry in those
    tiles, and exports those tiles.

    args:
        sink (str): type of data sink to export to. Viable options are: "asset" and "cloud".
        mode (str): either "subtidal" or "intertidal" for select type of bathymetry to export.
        geometry (ee.Geometry): geometry of the area of interest.
        zoom (int): zoom level of the to-be-exported tiles.
        start (ee.String): start date in YYYY-MM-dd format.
        stop (ee.String): stop date in YYYY-MM-dd format.
        scale Optional(float): scale of the product to be exported. Defaults tiler.zoom_to_scale(zoom).getInfo().
        crs (str): projection of the output image.
        buf_pix (int): buffer around the tile (in pixels).
        step_months (int): steps with which to roll the window over which the subtidal bathymetry
            is calculated.
        windows_months (int): number of months over which the bathymetry is calculated.
    """

    def create_year_window(year: ee.Number, month: ee.Number) -> ee.Dictionary:
        t: ee.Date = ee.Date.fromYMD(year, month, 1)
        d_format: str = "YYYY-MM-dd"
        return ee.Dictionary({
            "start": t.format(d_format),
            "stop": t.advance(window_months, 'month').format(d_format)
            })
    
    window_length: int = (parse(stop).year-parse(start).year)*12+(parse(stop).month-parse(start).month) # in months
    dates: ee.List = ee.List.sequence(parse(start).year, parse(stop).year-window_months/12).map(
        lambda year: ee.List.sequence(1, None, step_months, int((window_length-window_months)/step_months)+1).map(partial(create_year_window, year))
    ).flatten() # NOTE, still buggy, works for yearly composites. Not nice for end_date "2022-03-01"; error Date.fromYMD: Bad year/month/day: 2021/13/1.

    #dates = ee.List([dates.get(0)]) #ADJUSTED TO SELECT FIRST DATE ONLY
    
    # Get tiles
    tiled: ee.FeatureCollection = tiler.get_tiles_for_geometry(geometry, ee.Number(zoom))
    tiles: ee.FeatureCollection = tiled.map(lambda tile: tile.buffer(buf_pix*scale/111120, ee.ErrorMargin((buf_pix*scale*0.01)/111120, 'projected'), proj="EPSG:4326")) # fix for unequal buffer in EPSG3857: https://www.reddit.com/r/QGIS/comments/oo1jgh/buffer_points_on_epsg_4326_wgs_84/
    #tiledb: ee.FeatureCollection = tiled.map(lambda tile: tile.buffer(buf_pix*scale/111120, ee.ErrorMargin((buf_pix*scale*0.01)/111120, 'projected'), proj="EPSG:4326")) # ADJUSTED TO SELECT SINGLE TILE & fix for unequal buffer in EPSG3857: https://www.reddit.com/r/QGIS/comments/oo1jgh/buffer_points_on_epsg_4326_wgs_84/
    #tile: ee.Feature = ee.Feature(tiledb.filterMetadata("tx", "equals", tx_in).filterMetadata("ty", "equals", ty_in).first()) #11 #ADJUSTED TO SELECT SINGLE TILE
    #tiles: ee.FeatureCollection = ee.FeatureCollection(tile) #ADJUSTED TO SELECT SINGLE TILE

    if scale == None: scale: float = tiler.zoom_to_scale(zoom).getInfo() # not specified, defaults to pre-set float
    else: scale: scale # specified
    task_list: List[ee.batch.Task] = []
    #num_tiles: int = tiles.size().getInfo()
    #tile_list: ee.List = tiles.toList(num_tiles)

    for date in dates.getInfo():
        #print(date)
        if "subtidal" in mode:
            sdb_tiles: ee.ImageCollection = tiles.map(
                lambda tile: get_tile_subtidal_bathymetry(
                    tile=tile,
                    start=ee.String(date["start"]),
                    stop=ee.String(date["stop"])
                )#.clip(geometry) # clip individual tiles to match geometry of aoi
            )
        
        elif "intertidal" in mode:

            # filter tiles on possibility of having GTSM data. NOTE, the temporal extend takes the entire period of the composite, not the individual images. 
            # it is assumed if theres GTSM data, we will always find an image that is closeby. This assumption might not hold for all cases, investigate in the future (TODO)
            tiles = tiles.map(lambda tile: filter_tiles_close_to_GTSM(gtsm_col, tile=tile, start_date=ee.String(date["start"]), stop_date=ee.String(date["stop"])))
            tiles = tiles.filter(ee.Filter.gt('max_gtsm_size', 0))

            sdb_tiles: ee.ImageCollection = tiles.map(
                lambda tile: get_tile_intertidal_bathymetry(
                    tile=tile,
                    start=ee.String(date["start"]),
                    stop=ee.String(date["stop"])
                )#.clip(geometry)#.select('ndwi').rename('water_score') # clip individual tiles to match geometry of aoi, select ndwi and rename
            )

            # only export the ones with coupled GTSM data (i.e. depth values iso proxies)
            # NOTE, since we already filtered tiles above, we assume the code below is not needed anymore as sdb_tiles_up would be the same as sdb_tiles, investigate in the future (TODO)
            # if not the case, we might catch this by adding a SkipEmptyTiles in the batch image export function 
            #sdb_tile_prop: ee.FeatureCollection = sdb_tiles.select("gtsm_gebco_data_allempty") # ADDED filter GTSM data availability property
            #innerJoin = ee.Join.inner('primary', 'secondary') # ADDED construct join function
            #joined_tiles = innerJoin.apply(tiles, sdb_tile_prop, ee.Filter.equals(leftField= "id", rightField= "id")) # ADDED join tiles & sdb_tiles on id
            #joined_tiles_clean = joined_tiles.map(lambda feature: ee.Feature(feature.get('primary')).copyProperties(feature.get('secondary'))) # ADDED clean up joined tiles, keeping the tiles and only adding the selected property
            #tile_list: ee.FeatureCollection  = joined_tiles_clean.filterMetadata("gtsm_gebco_data_allempty", "equals", False) #ADDED TO UPDATE EXPORT FOR ONLY CALIBRATED IMAGES
            #sdb_tiles_up: ee.ImageCollection = sdb_tiles.filterMetadata("gtsm_gebco_data_allempty", "equals", False) #ADDED TO UPDATE EXPORT FOR ONLY CALIBRATED IMAGES

        num_tiles: int = tiles.size().getInfo() # tile_list #ADDED TO UPDATE EXPORT FOR ONLY CALIBRATED IMAGES
        #return sdb_tiles, tiles, tile_list#, sdb_tiles_up
        if num_tiles > 0:  #ADDED TO UPDATE EXPORT FOR ONLY CALIBRATED IMAGES
            tile_list: ee.List = tiles.toList(num_tiles)  # tile_list_up: ee.List = tile_list.toList(num_tiles) #ADDED TO UPDATE EXPORT FOR ONLY CALIBRATED IMAGES

            #return sdb_tiles, tiles, tile_list#, sdb_tiles_up, tile_list_up

            # Now export tiles
            export_sdb_tiles(
                sink=sink,
                tile_list=tile_list, # tile_list_up
                num_tiles=num_tiles,
                mode=mode,
                export_scale=scale,
                crs=crs,
                sdb_tiles=sdb_tiles, # sdb_tiles_up
                name_suffix=f"t{date['start']}_{date['stop']}_{scale}m3",
                task_list=task_list,
                overwrite=overwrite,
                bucket=bucket
            )

In [15]:
# TODO: clip tiles covering the AOI to pixels in the global coastal mask

In [18]:
# TODO: do intertidal mapping on single NDWI images as stated by Robyn & Maarten
# compute bathy and export to GCS
# when run submitted, check task progress at: https://code.earthengine.google.com/tasks or https://console.cloud.google.com/earth-engine/tasks?project=bathymetry

# export bathymetry based on standardized tiling practice
allfiles = os.listdir(os.path.join(main_fol, "00_miscellaneous\AOIs"))
tiled = []
for i in allfiles:
    if i.endswith("_adjusted.geojson") and not "CMEMS" in i and ("GER" in i):# or "BRA" in i or "ZAF" in i):
        print(i)
        with open(os.path.join(main_fol,"00_miscellaneous\AOIs",i)) as f:
            AoIjson = geojson.load(f)
        # try: # drawn polygon in this script
        #     AoI = Polygon(AoIjson["features"][0]["geometry"]["coordinates"]) 
        # except: # drawn in QGIS / ArcGIS and written to geojson there (client file)
        #     AoI = Polygon(AoIjson["features"][0]["geometry"]["coordinates"][0])
        # bounds = ee.Geometry.Polygon([[[a,b] for a, b in zip(*AoI.exterior.coords.xy)]])
        bounds = ee.Geometry.MultiPolygon(AoIjson["features"][0]["geometry"]["coordinates"])

        #sdb_tiles, tiles, tile_list = export_tiles(....)
        export_tiles(sink="cloud", mode=mode, geometry=bounds, zoom=zoomed_list[sel_tile], start=start_date, stop=stop_date, 
                     crs=crs, scale=scale, buf_pix=5, step_months=compo_int, window_months=compo_len, overwrite=True, bucket=bucket)

AOI_GER_WaddenSea_adjusted.geojson
Submitting task for tile:  intertidal_improved_100m/z11/x1063/y660/t2021-01-01_2022-01-01_100m3
Submitting task for tile:  intertidal_improved_100m/z11/x1064/y660/t2021-01-01_2022-01-01_100m3
Submitting task for tile:  intertidal_improved_100m/z11/x1065/y660/t2021-01-01_2022-01-01_100m3
Submitting task for tile:  intertidal_improved_100m/z11/x1066/y659/t2021-01-01_2022-01-01_100m3
Submitting task for tile:  intertidal_improved_100m/z11/x1066/y660/t2021-01-01_2022-01-01_100m3
Submitting task for tile:  intertidal_improved_100m/z11/x1067/y659/t2021-01-01_2022-01-01_100m3
Submitting task for tile:  intertidal_improved_100m/z11/x1068/y659/t2021-01-01_2022-01-01_100m3


In [19]:
#sdb_tiles.getInfo()

In [35]:
# store locally (from GCS) to visualize in QGIS / ArcGIS (can also download manually via Cloud Storage platform)

# create or check if local storage folder is present
if not os.path.exists(os.path.join(main_fol, output_fol)):
    os.makedirs(os.path.join(main_fol, output_fol))

# get file names
client = storage.Client()
ls = [blob for blob in client.list_blobs(bucket)] 

# downloading composites to a local folder while only keeping subtidal / intertidal folder (i.e. other nested folders are flattened)
check_files = []
for blob in ls:
    #if "tif" in blob.name:
    mode_fol = blob.name.split('/')[0]
    zoom_level = blob.name.split('/')[1]
    try: zoom_level_num = zoom_level.split("z")[1]
    except: zoom_level_num = zoom_level.split("z")[0]
    if mode_fol == mode and str(zoomed_list[sel_tile]) == zoom_level_num:
        file_name = "_".join(blob.name.split('/')[1:])
        check_files.append(file_name) #blob.name.split('/')[-1]
        if not os.path.exists(os.path.join(main_fol, output_fol, mode_fol)): # create subfolder
            os.makedirs(os.path.join(main_fol, output_fol, mode_fol))
        blob.download_to_filename(os.path.join(main_fol, output_fol, mode_fol, file_name))
        print('Stored: ', file_name) # check progress

# elaborate on possibility of storing locally
if len(check_files) == 0:
    print('Please enable GCS storeing of images first, before toggling on local storage option')

Stored:  z9_x113_y116_t2021-01-01_2022-01-01.tif
Stored:  z9_x113_y117_t2021-01-01_2022-01-01.tif
Stored:  z9_x114_y116_t2021-01-01_2022-01-01.tif
Stored:  z9_x114_y117_t2021-01-01_2022-01-01.tif
Stored:  z9_x114_y118_t2021-01-01_2022-01-01.tif
Stored:  z9_x115_y116_t2021-01-01_2022-01-01.tif
Stored:  z9_x115_y117_t2021-01-01_2022-01-01.tif
Stored:  z9_x115_y118_t2021-01-01_2022-01-01.tif
Stored:  z9_x163_y228_t2021-01-01_2022-01-01.tif
Stored:  z9_x163_y229_t2021-01-01_2022-01-01.tif
Stored:  z9_x164_y228_t2021-01-01_2022-01-01.tif
Stored:  z9_x164_y229_t2021-01-01_2022-01-01.tif
Stored:  z9_x189_y291_t2021-01-01_2022-01-01.tif
Stored:  z9_x189_y292_t2021-01-01_2022-01-01.tif
Stored:  z9_x190_y291_t2021-01-01_2022-01-01.tif
Stored:  z9_x190_y292_t2021-01-01_2022-01-01.tif
Stored:  z9_x263_y160_t2021-01-01_2022-01-01.tif
Stored:  z9_x263_y161_t2021-01-01_2022-01-01.tif
Stored:  z9_x263_y162_t2021-01-01_2022-01-01.tif
Stored:  z9_x264_y160_t2021-01-01_2022-01-01.tif
Stored:  z9_x264_y16